In [ ]:
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)
from tensorflow import keras
import tensorflow.compat.v2 as tf
from tensorflow import keras
import numpy as np
import pathlib
import random
import math
import time
from sklearn.model_selection import train_test_split



dataset_url = "/content/drive/MyDrive/Dataset"
data_dir = tf.keras.utils.get_file(origin=dataset_url,
                                   fname='dataset',
                                   untar=True)
data_dir = pathlib.Path(data_dir)


X = data_dir
y = "/content/drive/MyDrive/CoLA_Labels.csv"

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)




model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(250,250,1)),
  keras.layers.Reshape(target_shape=(250,250,1)), #0
  keras.layers.Conv2D(filters=64, kernel_size=(6,6), activation=tf.nn.relu, padding='SAME',kernel_initializer='he_uniform' ), #1 ##$$ If relu, then use he_initilization
  keras.layers.MaxPooling2D(pool_size=(2,2)), #2
  keras.layers.Dropout(0.2), #3
  keras.layers.Conv2D(filters=128, kernel_size=(5,5), activation=tf.nn.relu, padding='SAME',kernel_initializer='he_uniform' ), #4
  keras.layers.MaxPooling2D(pool_size=(2,2)), #5
  keras.layers.Dropout(0.2), #6
  keras.layers.Conv2D(filters=256, kernel_size=(4,4), activation=tf.nn.relu, padding='SAME',kernel_initializer='he_uniform' ), #7
  keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation=tf.nn.relu, padding='SAME',kernel_initializer='he_uniform' ), #8
  keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation=tf.nn.relu, padding='SAME',kernel_initializer='he_uniform' ), #9
  keras.layers.MaxPooling2D(pool_size=(2,2)), #10
  keras.layers.Dropout(0.2), #11
  keras.layers.Flatten(), #12
  keras.layers.Dense(500, activation=tf.nn.relu,kernel_initializer='he_uniform' ), #13
  keras.layers.Dense(128, activation=tf.nn.relu,kernel_initializer='he_uniform' ), #14
  keras.layers.Dropout(0.2), #15
  keras.layers.Dense(8, activation=tf.nn.softmax) #16
])

model.summary()


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
tic = time.time()

class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        val_acc = logs["val_accuracy"]
        if val_acc >= self.threshold:
            self.model.stop_training = True


my_callback = MyThresholdCallback(threshold=0.5)

model.fit(
  X_train,
  y_train,
  epochs=400,
  validation_data=(X_test, y_test),
  callbacks = [my_callback]
)

